In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from datetime import datetime, date

In [2]:
def grab_symbols():
    # Grab S&P Symbols from Wikipedia or local HTML File
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
    tickers = pd.read_html('./tickers.html')[0]
    tickers = tickers.Symbol.to_list()
    tickers = [i.replace('.','-') for i in tickers]
    return tickers 

print('Grab symbols function defined...')

Grab symbols function defined...


In [3]:
def yahoo_prices(tickers, current_day):
    # Return a DataFrame with dates, symbols and prices
    if not isinstance(tickers,list):
        return None

    years_back = 5
    end_dt = datetime.strptime(current_day, '%Y-%m-%d').date()
    start_dt = date(end_dt.year - years_back, end_dt.month, end_dt.day)

    data = yf.download(tickers, start=start_dt, end=end_dt)
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')    

Yahoo_prices function defined...


In [4]:
# tickers = grab_symbols()
# yahoo_df = yahoo_prices(tickers[0:3], '2023-01-30')
# sym_df = yahoo_df.loc[yahoo_df.Symbol == tickers[0]].copy()
# sym_df['NextOpen'] = sym_df.Open.shift(-1)
# sym_df['NextDate'] = sym_df.index
# sym_df['NextDate'] = sym_df.NextDate.shift(-1)
# sym_df

In [5]:
# Generate our indicators for Buy Signals for each of our assets
# df: contains one company (symbol) of data
def add_RSI_buy_signals(df):
    # Sanity check for our indicators to work
    if len(df) < 250:
        return None
    
    #df['MA200'] = df['Adj Close'].rolling(window=200).mean()
    df['NextOpen'] = df.Open.shift(-1)
    df['NextDate'] = df.index
    df['NextDate'] = df.NextDate.shift(-1)
    df['MA200'] = df['Close'].rolling(window=200).mean()
    #df['Price_change'] = df['Adj Close'].pct_change()
    df['Price_change'] = df['Close'].pct_change()
    df['Upmove']   = df['Price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['Price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg_up']   = df['Upmove'].ewm(span=19).mean()
    df['Avg_down']   = df['Downmove'].ewm(span=19).mean()
    df = df.dropna().copy()
    df['RS'] = df['Avg_up'] / df['Avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))
    #df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy_signal' ] = 'Yes'
    #df.loc[(df['Adj Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy_signal' ] = 'No'
    df.loc[(df['Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy_signal' ] = 'Yes'
    df.loc[(df['Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy_signal' ] = 'No'

    return df
    
print('add_RSI_buy_signals function defined....')

add_RSI_buy_signals function defined....


In [7]:
def order_simuator(df):
    max_hold_days = 10    # hold position for a maximum of 10 days
    stop_loss_percentage = -0.03
    target_percentage = 0.1
    # Order actions : Enter, Skip, Close, Stop, Target (a buy signal will be skipped if already in position)
    df['Order'] = ''

    in_position_start_index = None
    for index, row in df.iterrows():
        # Check for Enter position conditions
        if row.Buy_signal == 'Yes' and in_position_start_index is None:
            df.at[index, 'Order'] = 'Enter'
            in_position_start_index = index
        elif row.Buy_signal == 'Yes' and in_position_start_index is not None:
            df.at[index, 'Order'] = 'Skip'
        
        # Check for exit position conditions
        if in_position_start_index is not None:
            if (index - in_position_start_index).days >= max_hold_days:
                df.at[index, 'Order'] = 'Close'
                in_position_start_index = None
            elif index > in_position_start_index: 
                buy_price = df.at[in_position_start_index, 'Close']
                current_price = row['Close']
                percent_change = (current_price - buy_price) / buy_price
                if percent_change < stop_loss_percentage:
                    df.at[index, 'Order'] = 'Stop'
                    in_position_start_index = None
                if percent_change > target_percentage:
                    df.at[index, 'Order'] = 'Target'
                    in_position_start_index = None

    return df

print('Order simulator function defined....')

Order simulator function defined....


In [8]:
def extract_trades(df):
    # Loop through frame order information
    ## Grab Enter row to add to our trade dict
    ## Grab Sell row (Close, Target, Stop) to add to our ENTER row
    ## append to our trades
    ## Convert to a DataFrame
    # trade = {Id:999, Trade_seq: Tade_seq, Symbol: Ticker, Buy_dt: Date, Buy_Price: Price, Sell_dt: Date, Sell_Price: Price, Sell_Trigger: Close }
    # Id = symbol_Sort
    
    trades = []
    open_position = False
    trade_sequence = 1
    for index, row in df.iterrows():
        if not open_position: # Get the next Entry row
            if row.Order == 'Enter':
                trade_seq = trade_sequence
                symbol = row.Symbol
                buy_dt = row.NextDate
                buy_price = row.NextOpen
                open_position = True
        else: # Get the next Sell row (Close, Target, Stop)
            if row.Order in ['Close','Target','Stop']:
                sell_dt = row.NextDate
                sell_price = row.NextOpen
                sell_type = row.Order
                days_in_trade = (sell_dt - buy_dt).days
                profit = sell_price - buy_price
                percent_return = profit / buy_price
                trade = {'Id':f'{symbol}-{trade_seq}','Trade_seq':trade_seq,'Symbol':symbol,\
                         'Buy_dt':buy_dt,'Buy_Price':buy_price,\
                         'Sell_dt':sell_dt,'Sell_Price':sell_price,'Sell_type':sell_type,\
                         'Duration':days_in_trade,'profit':profit,'return':percent_return\
                        }
                trades.append(trade)
                open_position = False
                trade_sequence += 1
    trades_df = pd.DataFrame(trades)
    
    return trades_df

print('Extract Trades function defined....')  

Extract Trades function defined....


RUN SIMULATION FROM THIS POINT

In [9]:
# Grab symbols and get prices from yahoo
tickers = grab_symbols()
test_length = len(tickers)
all_assets = yahoo_prices(tickers[0:test_length], '2023-01-31')

[*********************100%***********************]  502 of 502 completed


In [11]:
# add indicators and buy signals, generate orders and store detailed results by date and symbol in an easily accessible list
multi_assets = []
len_tickers = len(tickers[0:test_length])
for i in range(len_tickers):
    frame = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
    frame = add_RSI_buy_signals(frame)
    if frame is not None:
        frame = order_simuator(frame)
        multi_assets.append(frame)

In [13]:
all_trades = None
for i in range(len(multi_assets)):
    frame = multi_assets[i]
    trades = extract_trades(frame)
    if all_trades is None:
        all_trades = trades
    else:    
        all_trades = pd.concat([all_trades, trades], axis=0, ignore_index=True)


wining_trades = len(all_trades.loc[all_trades.profit > 0])
losing_trades = len(all_trades.loc[all_trades.profit < 0])
target_trades = len(all_trades.loc[all_trades.Sell_type == 'Target'])
stop_trades   = len(all_trades.loc[all_trades.Sell_type == 'Stop'])
close_trades  = len(all_trades.loc[all_trades.Sell_type == 'Close'])
print(f'Wining trades: {wining_trades}')
print(f'Losing trades: {losing_trades}')
print(f'Targets: {target_trades}, Stops: {stop_trades}, Closes: {close_trades}')
print(f'Total profit : ${all_trades.profit.sum():0.2f}')
all_trades

Wining trades: 1345
Losing trades: 1292
Targets: 134, Stops: 881, Closes: 1624
Total profit : $1741.27


,Id,Trade_seq,Symbol,Buy_dt,Buy_Price,Sell_dt,Sell_Price,Sell_type,Duration,profit,return
0,MMM-1,1,MMM,2021-01-12 00:00:00-05:00,165.899994,2021-01-22 00:00:00-05:00,170.199997,Close,10,4.300003,0.025919
1,MMM-2,2,MMM,2021-06-17 00:00:00-04:00,196.800003,2021-06-29 00:00:00-04:00,196.199997,Close,12,-0.600006,-0.003049
2,AOS-1,1,AOS,2021-09-15 00:00:00-04:00,68.139999,2021-09-20 00:00:00-04:00,64.029999,Stop,5,-4.110001,-0.060317
3,AOS-2,2,AOS,2022-01-21 00:00:00-05:00,78.230003,2022-01-26 00:00:00-05:00,76.610001,Stop,5,-1.620003,-0.020708
4,AOS-3,3,AOS,2022-01-27 00:00:00-05:00,77.010002,2022-02-08 00:00:00-05:00,74.040001,Close,12,-2.970001,-0.038566
...,...,...,...,...,...,...,...,...,...,...,...
2634,ZTS-1,1,ZTS,2019-11-11 00:00:00-05:00,117.290001,2019-11-19 00:00:00-05:00,119.010002,Close,8,1.720001,0.014665
2635,ZTS-2,2,ZTS,2021-01-28 00:00:00-05:00,154.300003,2021-02-09 00:00:00-05:00,159.809998,Close,12,5.509995,0.035710
2636,ZTS-3,3,ZTS,2021-09-29 00:00:00-04:00,194.710007,2021-10-11 00:00:00-04:00,197.559998,Close,12,2.849991,0.014637
2637,ZTS-4,4,ZTS,2022-01-06 00:00:00-05:00,215.479996,2022-01-14 00:00:00-05:00,206.940002,Stop,8,-8.539993,-0.039632


In [ ]:
# Show results for one symbol (ticker) should have orders with [ENTER followed by a CLose/Target/Stop]
# Skip are entry signals that cannot be taken because already in a trade position for that symbol
#test_ticker = 2
#frame = multi_assets[test_ticker] 
#frame.loc[(frame.Order != '') | (frame.Buy_signal == 'Yes'), ['Symbol', 'Open', 'Close', 'Volume', 'Buy_signal', 'Order', 'NextDate', 'NextOpen']].tail(10)

# Calculating return on order : (sell_p - buy_p) / buy_p